In [3]:
from pathlib import Path
import polars as pl

file_path = Path.home() / "projects" / "okapi" / "towns" / "towns_step2_original_frame.xlsx"
file_path.exists()

df = pl.read_excel(file_path)
df

Lfd. Nr.,Land,RB,Kreis,Verb,Gem,Stadt,Postleitzahl,Fläche in km2,insgesamt,männlich,weiblich,je km2
i64,str,str,str,str,str,str,str,f64,i64,i64,i64,i64
1,"""11""","""0""","""00""","""0000""","""000""","""Berlin, Stadt""","""10178""",891.12,3782202,1860115,1922087,4244
2,"""02""","""0""","""00""","""0000""","""000""","""Hamburg, Freie und Hansestadt""","""20038""",755.09,1910160,936740,973420,2530
3,"""09""","""1""","""62""","""0000""","""000""","""München, Landeshauptstadt""","""80313""",310.7,1510378,734925,775453,4861
4,"""05""","""3""","""15""","""0000""","""000""","""Köln, Stadt""","""50667""",405.02,1087353,527728,559625,2685
5,"""06""","""4""","""12""","""0000""","""000""","""Frankfurt am Main, Stadt""","""60311""",248.31,775790,382226,393564,3124
…,…,…,…,…,…,…,…,…,…,…,…,…
2055,"""16""","""0""","""75""","""5013""","""127""","""Ziegenrück, Stadt""","""07924""",8.25,615,303,312,75
2056,"""03""","""3""","""54""","""5403""","""021""","""Schnackenburg, Stadt""","""29493""",23.7,521,259,262,22
2057,"""16""","""0""","""71""","""5053""","""061""","""Neumark, Stadt""","""99439""",8.67,482,253,229,56


In [ ]:
import pandas as pd

# 1. Read the Excel file (header row at line 0 by default)
df = pd.read_excel(file_path, header=0)

# 2. Cast selected columns to pandas’ nullable integer dtype “Int64”
int_cols = [
    "Lfd. Nr.",       # index
    "Land",           # state code
    "RB",             # region code
    "Kreis",          # district code
    "Verb",           # administrative union code
    "Gem",            # municipality code
    "Postleitzahl"    # postal code
]
df[int_cols] = df[int_cols].astype("Int64")

# 3. Rename columns
df = df.rename(columns={
    "Lfd. Nr.":      "lfd_nr",
    "Land":          "land",
    "RB":            "rb",
    "Kreis":         "kreis",
    "Verb":          "verb",
    "Gem":           "gem",
    "Stadt":         "stadt",
    "Postleitzahl":  "plz",
    "Fläche in km2": "flaeche_km2",
    "insgesamt":     "pop_insgesamt",
    "männlich":      "pop_maennlich",
    "weiblich":      "pop_weiblich",
    "je km2":        "pop_density"
})

df[["stadt_name", "stadt_typ"]] = df["stadt"].str.split(", ", n=1, expand=True)

# 2. Drop the original column
df = df.drop(columns=["stadt"])
df = df.drop(columns=["stadt_typ"])

df = df.drop(columns=["lfd_nr", "land", "rb", "kreis", "verb", "gem", "plz"])

df = pl.from_pandas(df)
df

flaeche_km2,pop_insgesamt,pop_maennlich,pop_weiblich,pop_density,stadt_name
f64,i64,i64,i64,i64,str
891.12,3782202,1860115,1922087,4244,"""Berlin"""
755.09,1910160,936740,973420,2530,"""Hamburg"""
310.7,1510378,734925,775453,4861,"""München"""
405.02,1087353,527728,559625,2685,"""Köln"""
248.31,775790,382226,393564,3124,"""Frankfurt am Main"""
…,…,…,…,…,…
8.25,615,303,312,75,"""Ziegenrück"""
23.7,521,259,262,22,"""Schnackenburg"""
8.67,482,253,229,56,"""Neumark"""


In [12]:
p = Path.home() / "projects" / "okapi" / "towns" / "towns_step3.csv"
df.write_csv(p, include_header=True)